In [1]:
model = "llama3.2"

#### Task 1: Create a Simple Chain for Summarization


**Objective:**

Build a LangChain chain that can summarize a given text.

**Task Description:**

- Create a LLMChain using with a Ollama model.
- Define a prompt template for summarization.
- Run the chain with a sample text and print the summary.
- Add model output streaming.
- Run the chain with a sample text and print the summary.


In [2]:
from langchain_ollama.chat_models import ChatOllama
from langchain_core.prompts import ChatPromptTemplate

# Load the Ollama model
llm = ChatOllama(model=model)

# Define the prompt template
summarization_prompt = ChatPromptTemplate([
    ("system", "Your task is to summarize a text in one sentence."),
    ("human", "{text}"),
])

# Create the LLMChain
summarization_chain = summarization_prompt | llm 

# Run the chain with a sample text
text = """Over the last decade, deep learning has evolved massively to process and generate unstructured data like text, images, and video. 
These advanced AI models have gained popularity in various industries, and include large language models (LLMs). 
There is currently a significant level of fanfare in both the media and the industry surrounding AI,
and there’s a fair case to be made that Artificial Intelligence (AI), with these advancements,
is about to have a wide-ranging and major impact on businesses, societies, and individuals alike.
This is driven by numerous factors, including advancements in technology, high-profile applications, 
and the potential for transfor- mative impacts across multiple sectors."""

summary = summarization_chain.invoke(text)
print(summary.content)

Deep learning has revolutionized the processing and generation of unstructured data through advanced AI models like large language models, poised to have a significant impact on businesses, societies, and individuals in various industries.


In [4]:
# Stream the chain output
for chunk in summarization_chain.stream({"text": text}):
    print(chunk.content, end="", flush=True)#

Deep learning has made significant strides over the last decade, enabling advanced AI models such as large language models to process and generate unstructured data with widespread potential impacts on businesses, societies, and individuals.

#### Task 2: Chain with Tool Usage (Simple Math Tool)

**Objective:**

Create a LangChain chain that uses a simple math tool to perform calculations.

**Task Description:**

- Define a function as tool which multiplies two integer values and return the result.
- Create a chain
- Print the result of the calculation.

In [5]:
from langchain_core.tools import tool

# Create custom tool
@tool
def multiply(first_int: int, second_int: int) -> int:
    """Multiply two integers together."""
    return first_int * second_int

print(multiply.name)
print(multiply.description)
print(multiply.args) # -> definition of tool arguments

# Invoke custom tool
multiply.invoke({"first_int": 4, "second_int": 5})

multiply
Multiply two integers together.
{'first_int': {'title': 'First Int', 'type': 'integer'}, 'second_int': {'title': 'Second Int', 'type': 'integer'}}


20

In [6]:
# Load the Ollama model
llm = ChatOllama(model=model)

# Use bind_tools to pass the definition of our tool in as part of each call to the model, so that the model can invoke the tool
llm_with_tools = llm.bind_tools([multiply])

# When the model invokes the tool, this will show up in the AIMessage.tool_calls attribute of the output -> extract tool parameters from input text
msg = llm_with_tools.invoke("whats 5 times forty two")
print(msg.tool_calls)

[{'name': 'multiply', 'args': {'first_int': '42', 'second_int': '5'}, 'id': 'ae555d8a-7fd4-4aa0-a11f-e772da22bea1', 'type': 'tool_call'}]


In [7]:
# Create the chain: pass the extracte tool parameters from the input text to the tool -> extract the arguments of the first tool_call
chain_with_tools = llm_with_tools | (lambda x: x.tool_calls[0]["args"]) | multiply

# Run chain
chain_with_tools.invoke("whats 5 times forty two")

210

#### Task 3: Agent with Tool Usage (Two Tools)

**Objective:** 

Create a LangChain agent that uses two tools to perform tasks.

**Task Description:**

- Define prompt template.
- Define tools.
- Create an Agent using the Ollama model, prompt template and tools.
- Run the agent with a prompt that requires one or both tools.
- Observe how the agent uses the tools to complete the task.

In [8]:
from langchain.agents import AgentExecutor, create_tool_calling_agent

# Load the Ollama model
llm = ChatOllama(model=model)

agent_prompt = ChatPromptTemplate.from_messages([
    ("system", "you're a helpful assistant"), 
    ("human", "{user_input}"), 
    ("placeholder", "{agent_scratchpad}"),
])


# Custom math tools
@tool
def add(first_int: int, second_int: int) -> int:
    "Add two integers."
    return first_int + second_int


@tool
def exponentiate(base: int, exponent: int) -> int:
    "Exponentiate the base to the exponent power."
    return base**exponent


tools = [add, exponentiate]

In [9]:
# Construct the tool calling agent
agent_with_tools = create_tool_calling_agent(llm, tools, agent_prompt)

# Create an agent executor by passing in the agent and tools
agent_executor_with_tools = AgentExecutor(agent=agent_with_tools, tools=tools, verbose=True, return_intermediate_steps=False)

In [13]:
agent_executor_with_tools.invoke(
    {
        "user_input": "Take 3 to the fifth power then add that 12."
    }
)



> Entering new AgentExecutor chain...

Invoking: `exponentiate` with `{'base': '3', 'exponent': '5'}`


243
Invoking: `add` with `{'first_int': '11', 'second_int': '12'}`


23The result of 3 to the fifth power is 243. Adding 12 to this gives us a final answer of 255.

> Finished chain.


{'user_input': 'Take 3 to the fifth power then add that 12.',
 'output': 'The result of 3 to the fifth power is 243. Adding 12 to this gives us a final answer of 255.'}

#### [Optional] Task 4: Enhance Agent with Memory

**Objective:**

Eenhance the agent from Task 3 with memory to improve its context awareness and ability to maintain state.

**Instructions:**

- Create a ConversationBufferMemory to store chat history.
- Modify the agent to use the memory to inform its responses.
- Run the agent with a series of prompts that require context or state to be maintained.
- Observe how the agent's responses improve with the addition of memory.

In [15]:
from langchain.memory import ConversationBufferMemory
from langchain_core.prompts import MessagesPlaceholder

# Load the Ollama model
llm = ChatOllama(model=model)

# Define memory object for conversation history
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True, output_key="output")

# Add history placeholder to prompt
agent_prompt_with_memory = ChatPromptTemplate.from_messages([
    ("system", "you're a helpful assistant"), 
    MessagesPlaceholder("chat_history"),
    ("human", "{user_input}"), 
    ("placeholder", "{agent_scratchpad}"),
])

# Construct the tool calling agent
agent_with_tools_and_memory = create_tool_calling_agent(llm, tools, agent_prompt_with_memory)

# Create an agent executor by passing in the agent and tools
agent_executor_with_tools_and_memory = AgentExecutor(agent=agent_with_tools_and_memory, tools=tools, memory=memory, verbose=True, return_intermediate_steps=True)

In [21]:
question = "Take 3 to the fifth power then add that 12?"
ai_msg_1 = agent_executor_with_tools_and_memory.invoke({"user_input": question})
print(ai_msg_1["output"])

second_question = "Explain how you have calculated the result."
ai_msg_2 = agent_executor_with_tools_and_memory.invoke({"user_input": second_question})
print(ai_msg_2["output"])



> Entering new AgentExecutor chain...

Invoking: `exponentiate` with `{'base': '3', 'exponent': '5'}`


243
Invoking: `add` with `{'first_int': '9', 'second_int': '12'}`


21The result of 3 to the fifth power is 243, and when you add that to 12, the final answer is 255.

> Finished chain.
The result of 3 to the fifth power is 243, and when you add that to 12, the final answer is 255.


> Entering new AgentExecutor chain...

Invoking: `exponentiate` with `{'base': 3, 'exponent': 5}`


243
Invoking: `add` with `{'first_int': 243, 'second_int': 12}`


255I calculated the result by following the order of operations (PEMDAS):

1. Exponentiate 3 to the fifth power, which means multiplying 3 by itself 5 times: `3^5 = 3 × 3 × 3 × 3 × 3 = 243`
2. Add 12 to the result: `243 + 12 = 255`

Therefore, the final answer is 255.

> Finished chain.
I calculated the result by following the order of operations (PEMDAS):

1. Exponentiate 3 to the fifth power, which means multiplying 3 by itself 5 times: 